# Mini projet Qualité de Données : Détections des doublons
## ***Christophe COMPAIN / Sander COHEN***

### Objectif et Données Disponibles
L'objectif du projet est d'identifier les logiciels vendus sur les deux plateformes.

Pour ce faire, nous disposons des données pour chacune des plateformes isolément, respectivement dans les fichiers ***Company1.csv*** et ***Company2.csv***. 

### Import packages, Variables Globales et import csv

In [155]:
import pandas as pd
import nltk
import time
import numpy as np
import math
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scohe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\scohe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [156]:
path = "D:\\OneDrive - Université Paris-Dauphine\\Bureau\\Cours Master\\12-Qualité de Données\\\Projet\\mini-projet\\"
file1= "Data\\Company1.csv" #"SampleData\\Sample_Company1.csv"
file2= "Data\\Company2.csv" #"SampleData\\Sample_Company2.csv"
real= "Data\\Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"

In [157]:
company1 = pd.read_csv(path+file1, encoding = "ISO-8859-1")
company2 = pd.read_csv(path+file2, encoding = "ISO-8859-1")
ground_truth_matches = pd.read_csv(path+real, encoding = "ISO-8859-1")

### Exploration des données

In [158]:
company1.head(5)

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops and desktops,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonprofits 2007,peachtree premium accounting for nonprofits 2007 is the affordable easy to use accounting solution that provides you with donor/grantor management. if you're like most nonprofit organizations you're constantly striving to maximize each and every dollar of your annual operating budget. financial reporting by programs and funds advanced operational reporting and the rock-solid core accounting features that have made peachtree the choice of hundreds of thousands of small businesses. the result is an accounting solution tailor-made for the challenges of operating a nonprofit organization. keep an audit trail to record and report on any changes made to your transactions improve data integrity with prior period locking archive your organization's data for snap shots of your data before you closed your year set up individual user profiles with password protection peachtree restore wizard restores all backed-up data files plus web transactions and customized forms includes all standard accounting features - general ledger accounts receivable &accounts payable; payroll solutions; time & billing and job costing; fixed assets; analysis and reporting & customization easily convert from quickbooks,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning products (win me nt 2000 xp),carry-a-tune technologies,99.99


In [159]:
company2.head(5)

,id,name,description,manufacturer,price
0,11125907881740407428,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,11538923464407758599,superstart! fun with reading & writing!,fun with reading & writing! is designed to help kids learn to read and write better through exercises puzzle-solving creative writing decoding and more!,NaN,8.49
2,11343515411965421256,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for retailers who need basic inventory sales and customer tracking.,intuit,637.99
3,12049235575237146821,math missions: the amazing arcade adventure (grades 3-5),save spectacle city by disrupting randall underling's plan to drive all the stores out of business and take over the city. solve real-world math challenges in the uniquely entertaining stores and make theme successful again.,NaN,12.95
4,12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from production studio premium or standard,adobe software,805.99


In [160]:
ground_truth_matches.head(5)

,idCompany1,idCompany2
0,b000jz4hqo,18441480711193821750
1,b00004tkvy,18441110047404795849
2,b000g80lqo,18441188461196475272
3,b0006se5bq,18428750969726461849
4,b00021xhzw,18430621475529168165


#### Observation d'un premier duplicat

In [161]:
company1[company1.id == ground_truth_matches.idCompany1[1]]

,id,title,description,manufacturer,price
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.0


In [162]:
company2[company2.id == ground_truth_matches.idCompany2[1]]

,id,name,description,manufacturer,price
1881,18441110047404795849,the beginners bible: noah's ark activity center: activity center,NaN,NaN,9.95


In [163]:
##retraitement des prix
def retreatprice(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    return float(re.sub("[^0-9.]", " ",str(texte)))
company1['price2'] = company1['price'].apply(retreatprice)
company2['price2'] = company1['price'].apply(retreatprice)

In [164]:
stop_words = set(nltk.corpus.stopwords.words('english'))  
stop_words.update("r")

def prep(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    texte = re.sub("[^a-zA-Z0-9_]", " ",str(texte)).lower()
    #tokenization par mot
    tokens = nltk.word_tokenize(texte)
    #supreesion des stopwords
    filtered_tokens = [w for w in tokens if not w in stop_words]
#    # Stemming
#    texte = [nltk.stem.SnowballStemmer('english').stem(w) for w in filtered_tokens]
    # Lemmatization
    texte = [nltk.stem.WordNetLemmatizer().lemmatize(w) for w in filtered_tokens]
    #remise sous forme d'une string
    return " ".join(texte)
        

In [165]:
company1['Company']="company1"
company1=company1.rename(columns={"title": "name"})
company2['Company']="company2"


In [166]:
company1_rich = company1[(company1.price2>1000.00) | (company1.price2==0)]
company2_rich = company2[(company2.price2>1000.00) | (company2.price2==0)]
len(company1_rich)

255

In [167]:
corpus_rich = pd.concat([company1_rich, company2_rich],sort=False,ignore_index=True)
corpus['name'] = corpus['name'].fillna(' ')
corpus['manufacturer'] = corpus['manufacturer'].fillna(' ')
corpus['description'] = corpus['description'].fillna(' ')
corpus['full data']=corpus['manufacturer'].apply(prep) + ' ' + corpus['name'].apply(prep) # + ' ' + corpus['description'].apply(prep)#corpus['manufacturer'] + ' ' + corpus['name'] + ' ' + corpus['description']

#corpus.reset_index(drop=True)
len(corpus)
corpus.tail()

,id,name,description,manufacturer,price,price2,Company,full data
505,7683929839876581168,peachtree(r) compatible continuous checks-accounting for windows 8.0-2004 version,compatible with peachtree(r) accounting for windows 8.0-2004 version software.continuous check size: 9 1/2in. x 7in..,,91.59,0.00,company2,peachtree compatible continuous check accounting window 8 0 2004 version
506,7213428761907802812,shrek the third for pc,save far far away as you adventure as shrek and his friends on a quest from the movie and beyond in this hilarious twisted fairy tale. play as your favorite characters and new friends from the movie each with unique abilities and outlandish moves. ...,,19.99,1599.00,company2,shrek third pc
507,5514066857425250107,laplink software pafglplk0b000p0rtpen - laplink laplink gold v.12.0 - complete product - file transfer - 3 device 1 user - complete product - standard - english - pc,laplink software pafglplk0b000p0rtpen : connectivity communication virus-scanning and remote control - all in one product. laplink gold 12 is the new standard for secure remote access and virus-free file transfer and synchronization. laplink gold ...,,98.97,0.00,company2,laplink software pafglplk0b000p0rtpen laplink laplink gold v 12 0 complete product file transfer 3 device 1 user complete product standard english pc
508,14832301369335122505,peachtree(r) complete iii,guaranteed to work with peachtree(r) complete iii for windows(r)!statements. get your customers to pay promptly.invoices. plenty of room for describing and listing products/services and amounts.multipurpose forms. hundreds of uses. ideal for ...,,72.99,0.00,company2,peachtree complete iii
509,5835993203508446584,sony imz-rs301 imzrs301 additional client upgrade cctv software,imz-rs301 additional camera license upgrade for real shot manager v3 software package the sony imz-rs301 is an additional camera licence upgrade for the real shot manager v3 ip video monitoring software. this license agreement allows users to add ...,,374.95,3314.09,company2,sony imz rs301 imzrs301 additional client upgrade cctv software


In [168]:
corpus_rich['full data']

0      broderbund clickart 950 000 premier image pack dvd rom           
1      computer associate ca international arcserve lap desktop oem 30pk
                                     ...                                
508     peachtree complete iii                                          
509     sony imz rs301 imzrs301 additional client upgrade cctv software 
Name: full data, Length: 510, dtype: object

In [207]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.01,sublinear_tf=True) #ngram_range=(1),
vectors = vectorizer.fit_transform(corpus_rich['full data'])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
#denselist = dense.tolist()
#df = pd.DataFrame(denselist, columns=feature_names)


In [208]:
#feature_names
dense.shape #.head() 4589 rows × 12391 columns

(510, 3884)

In [209]:
company1_rich

,id,name,description,manufacturer,price,price2,Company
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00,0.00,company1
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops and desktops,computer associates,0.00,0.00,company1
...,...,...,...,...,...,...,...
1341,b000jx84va,instant immersion hawaiian audio,NaN,topics entertainment,0.00,0.00,company1
1358,b000cs3s2c,flash remoting 1 alp ret eng cd 2u,- marketing information: macromedia flash remoting mx provides the connection between macromedia flash and your web application server making it fast and easy to create rich internet applications. with its powerful yet simple programming model you can easily integrate rich macromedia flash content with applications built using macromedia coldfusion mx microsoft .net java and soap-based web services. product information - software sub type: web designing/publishing - software name: macromedia flash remoting mx - complete product - features and benefits: lower costs and accelerate development: - easily access business logic - simplify your team workflow - streamline your testing and debugging - continue to use xml build robust scalable rich internet applications: - deploy in your native application server environment - promote well-designed applications - simplify recordset handling - use built-in application server features connect flash to leading application server technologies: - use your existing web application server - easily access web services from macromedia flash - deploy consistently across platforms and devices - language support: international english - platform support: pc - platform support: ultrasparc license information - license type: complete product - license pricing: standard - license quantity: 2 processor,adobe,3314.09,3314.09,company1


In [217]:
##tfidf
number_of_matches = 0
matches=[]
start = time.process_time()
for i in range(len(company1_rich)):
    tokens1name = nltk.word_tokenize(str(company1_rich.iloc[i,3]) + ' ' + str(company1_rich.iloc[i,1]))
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=2))
    for j in range(len(company2_rich)):
        tokens2name = nltk.word_tokenize(str(company2_rich.iloc[j,3]) + ' ' + str(company2_rich.iloc[j,1])  )
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=2))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
#        try :
#            similarity = np.dot(dense[i],np.transpose(dense[len(company1_rich)+j])).item(0)/math.sqrt(np.dot(dense[i],np.transpose(dense[i])).item(0) * np.dot(dense[len(company1_rich)+j],np.transpose(dense[len(company1_rich)+j])).item(0))
#        except : 
#            similarity = 0
        if jd_ng1_ng2_name<0.7:#((similarity > 0.4)) :# or name_score<=1) :
            number_of_matches = number_of_matches +1
            matches.append((company1_rich.iloc[i,0],company2_rich.iloc[j,0]))
print("Number of matches: {}".format(number_of_matches))
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']
end = time.process_time()
print("Processing time: {}".format(end - start))
print("Number of true positives: {}".format(len(true_positives)))
print("Number of false positives: {}".format(len(false_positives)))
print("Number of false negatives: {}".format(len(false_negatives)))
precision = len(true_positives)/(len(true_positives)+ len(false_positives))
print("Precision: {}".format(precision))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
print("Recall: {}".format(recall))
f_measure = 2*(precision*recall)/(precision+recall)
print("F measure: {}".format(f_measure))

Number of matches: 16
Processing time: 14.625
Number of true positives: 6
Number of false positives: 10
Number of false negatives: 1294
Precision: 0.375
Recall: 0.004615384615384616
F measure: 0.009118541033434652


In [176]:
#sans retraitement prix
Number of matches: 1309
Processing time: 820.0
Number of true positives: 727
Number of false positives: 582
Number of false negatives: 573
Precision: 0.5553857906799083
Recall: 0.5592307692307692
F measure: 0.5573016481410501

SyntaxError: invalid syntax (<ipython-input-176-4c44c09c7517>, line 2)

In [ ]:
base_false_negatives =false_negatives.merge(corpus.loc[corpus['Company'] == 'company1']
                                            .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                            .rename(columns = {'id': 'idCompany1','description': 'descr1',
                                                               'price': 'price1','full data': 'full data1'}
                                                    , inplace = False)
                                            , how='inner', on='idCompany1').merge(corpus.loc[corpus['Company'] == 'company2']
                                                                                  .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                                                                  .rename(columns = {'id': 'idCompany2', 
                                                                                                     'description': 'descr2', 
                                                                                                     'price': 'price2', 
                                                                                                     'full data': 'full data2'}, inplace = False)
                                                                                  , how='inner', on='idCompany2')

In [135]:
base_false_positives =false_positives.merge(corpus.loc[corpus['Company'] == 'company1']
                                            .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                            .rename(columns = {'id': 'idCompany1','description': 'descr1',
                                                               'price': 'price1','full data': 'full data1'}
                                                    , inplace = False)
                                            , how='inner', on='idCompany1').merge(corpus.loc[corpus['Company'] == 'company2']
                                                                                  .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                                                                  .rename(columns = {'id': 'idCompany2', 
                                                                                                     'description': 'descr2', 
                                                                                                     'price': 'price2', 
                                                                                                     'full data': 'full data2'}, inplace = False)
                                                                                  , how='inner', on='idCompany2')

In [136]:
base_false_positives


,idCompany1,idCompany2,Exist,descr1,price1,full data1,descr2,price2,full data2
0,b000j4k804,16690738478999740250,right_only,instant immersion spanish (audio book) (audio book),0.0,topic entertainment instant immersion spanish audio book,from madrid to mexico city now you can join the conversation with instant immersion spanish audio deluxe the dynamic language learning system comprising recorded audio instruction and printed lesson reinforcement.,0.0,topic entertainment instant immersion spanish audio deluxe
1,b000ivl050,17962667015870936885,right_only,instant immersion german audio (audio book),0.0,topic entertainment instant immersion german audio,business trip to berlin? holiday in hamburg? from munich to munster now you can join the conversation with instant immersion german v2.0 the dynamic 5-disc language learning system on cd-rom. system requirements: pc: windows 98,0.0,topic entertainment instant immersion german 2 0


In [32]:
corpus = pd.concat([company1, company2],sort=False,ignore_index=True)
corpus['name'] = corpus['name'].fillna(' ')
corpus['manufacturer'] = corpus['manufacturer'].fillna(' ')
corpus['description'] = corpus['description'].fillna(' ')
corpus['full data']=corpus['manufacturer'].apply(prep) + ' ' + corpus['name'].apply(prep) # + ' ' + corpus['description'].apply(prep)#corpus['manufacturer'] + ' ' + corpus['name'] + ' ' + corpus['description']

#corpus.reset_index(drop=True)
len(corpus)
corpus.tail()

,id,name,description,manufacturer,price,Company,full data
4584,14872602878188858026,jumpstart(r) advanced 1st grade,prepare your child for the 1st grade and beyond with jumpstart advanced 1st grade. this 4-cd set ensures the mastery of fundamental skills in subjects like reading math and science. includes the all new pet playground where kids care for and play ...,,NaN,company2,jumpstart advanced 1st grade
4585,14916162814320983138,ibm(r) viavoice(r) advanced edition 10,ibm viavoice advanced edition release 10 is a feature-rich voice-recognition product designed for experienced pc users. windows xp home and professional windows 98se and windows me are the supported operating systems for advanced edition. viavoice ...,,NaN,company2,ibm viavoice advanced edition 10
4586,14974113209571399013,xbox 360: gears of war,as marcus fenix you fight a war against the immense locust horde which not only out powers but outnumbers you your squad and the entire force of the coalition of ordered governments. gears of war is the only game to blend a deep and disturbing ...,,NaN,company2,xbox 360 gear war
4587,14986935400648190776,documents to go premium 7.0,this pda software enables you to use your documents such as word excel powerpoint pictures and pdf files on your handheld computer. view edit and create word excel and powerpoint files and view pdf graphics and image files. leave your laptop ...,,NaN,company2,document go premium 7 0
4588,14996991014087320062,microsoft(r) picture it! digital image pro 9.0,picture it! digital image pro puts you in control of your photos through a unique combination of powerful digital imaging tools with helpful wizards and professional-quality photo projects. picture it! digital image pro offers advanced tools that ...,,NaN,company2,microsoft picture digital image pro 9 0


In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.01,sublinear_tf=True) #ngram_range=(1),
vectors = vectorizer.fit_transform(corpus['full data'])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
#denselist = dense.tolist()
#df = pd.DataFrame(denselist, columns=feature_names)


In [19]:
base_false_negatives

,idCompany1,idCompany2,Exist,descr1,price1,full data1,descr2,price2,full data2
0,b0006se5bq,18428750969726461849,left_only,singing coach unlimited - electronic learning ...,99.99,carry tune technology singing coach unlimited,learn to sing with the help of a patented real...,82.5,singing coach unlimited electronic learning p...
1,b00021xhzw,18430621475529168165,left_only,upgrade only; installation of after effects st...,499.99,adobe adobe effect professional 6 5 upgrade st...,adobe after effects pb 6.5 win upgrade.standar...,507,adobe software 22070152 effect 6 5 pbupgrd
2,b0000dbykm,18363672170449359273,left_only,in mia's math adventure: just in time children...,19.99,kutoka mia math adventure time,kutoka interactive 61208 : mia s math adventur...,18.97,kutoka interactive 61208 mias math adventure ...
3,b00029bqa2,18431790997623472871,left_only,disney's 1st & 2nd grade bundle will help your...,14.99,disney disney 1st 2nd grade bundle pixar 1st g...,disney learning 1st & 2nd features an all-star...,13.61,disney learning 1st 2nd grade win
4,b00006hvvo,3409784122469217433,left_only,today enterprises and service providers face i...,0,sonic system inc upg sgms 1000 incremental node,today enterprises and service providers face i...,62920.89,sonicwall gm 1000n incremental lic upg
...,...,...,...,...,...,...,...,...,...
568,b0009x6qew,18305953820307083661,left_only,with instant immersion italian deluxe you'll g...,29.99,topic entertainment instant immersion italian ...,your first class journey to a new ready to com...,31.06,topic entertainment instant immersion italian...
569,b000cs3s2c,17715266298340609214,left_only,- marketing information: macromedia flash remo...,3314.09,adobe flash remoting 1 alp ret eng cd 2u,macromedia flash remoting mx provides the conn...,1928.09,adobe system inc flash remoting 1 0 net java ...
570,b00005bigp,17738870551709614779,left_only,,9.99,school zone shape,shapes challenges children to identify and cre...,9.45,school zone interactive shape track software
571,b000p9cr66,17724977097925207764,left_only,mediarecover gives you the ability to recover ...,29.99,aladdin system mediarecover,mediarecover retrieves your lost photos audio ...,26.14,allume system inc mediarecover


In [69]:
pd.set_option('display.max_colwidth', -1)
pd.set_option("max_rows", 5)#None)

def retreatprice(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    return float(re.sub("[^0-9.]", " ",str(texte)))

company2['test']=company2['price'].apply(retreatprice)
company2[['price','test']].sort_values(by=['test'])

0      0.00     
202    0.00     
       ...      
121    42166.22 
130    101515.55
Name: price, Length: 1363, dtype: float64